# Trail Experiment to do  Machine Learning on Blockchain(BlockFL)


#### Install

To run this project on Google Colab, please install these following python packages:

In [1]:
!pip install Web3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 501 kB 34.2 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 103 kB 57.1 MB/s 
     |████████████████████████████████| 82 kB 936 kB/s 
     |████████████████████████████████| 1.1 MB 59.6 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 235 kB 59.2 MB/s 
     |████████████████████████████████| 2.3 MB 52.3 MB/s 
     |████████████████████████████████| 1.6 MB 64.0 MB/s 
     |████████████████████████████████| 1.9 MB 41.8 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42725 sha256=c7fac6dbbebf7f0794411e453abae1827c1410cbb1e0a8e49d9bc4849745654b
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
  Created wheel for varint: filename=varint-1.0.2-py3-no

In [1]:
!pip install eth_tester

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 70 kB 6.9 MB/s 
     |████████████████████████████████| 102 kB 46.7 MB/s 
  Attempting uninstall: eth-typing
    Found existing installation: eth-typing 2.3.0
    Uninstalling eth-typing-2.3.0:
      Successfully uninstalled eth-typing-2.3.0
  Attempting uninstall: eth-utils
    Found existing installation: eth-utils 1.9.5
    Uninstalling eth-utils-1.9.5:
      Successfully uninstalled eth-utils-1.9.5
  Attempting uninstall: rlp
    Found existing installation: rlp 2.0.1
    Uninstalling rlp-2.0.1:
      Successfully uninstalled rlp-2.0.1
  Attempting uninstall: eth-keys
    Found existing installation: eth-keys 0.3.4
    Uninstalling eth-keys-0.3.4:
      Successfully uninstalled eth-keys-0.3.4
  Attempting uninstall: eth-rlp
    Found existing installation: eth-rlp 0.2.1
    Uninstalling eth-rlp-0.2.1:
      Successfully uninstalled eth-rlp-0.2.1

In [2]:
!pip install "web3[tester]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached eth_typing-2.3.0-py3-none-any.whl (6.2 kB)
  Using cached eth_utils-1.10.0-py3-none-any.whl (24 kB)
  Using cached eth_abi-2.2.0-py3-none-any.whl (28 kB)
  Using cached eth_account-0.5.9-py3-none-any.whl (101 kB)
  Using cached eth_rlp-0.2.1-py3-none-any.whl (5.0 kB)
     |████████████████████████████████| 69 kB 6.8 MB/s 
  Using cached eth_keys-0.3.4-py3-none-any.whl (21 kB)
  Using cached rlp-2.0.1-py2.py3-none-any.whl (20 kB)
     |████████████████████████████████| 276 kB 55.4 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
  Using cached eth_keyfile-0.5.1-py3-none-any.whl (8.3 kB)
     |████████████████████████████████| 829 kB 45.4 MB/s 
  Created wheel for pyethash: filename=pyethash-0.1.27-cp37-cp37m-linux_x86_64.whl size=51995 sha256=edac3fb80cb817dc0a465f6200162c127fae3dd4c9fa287f60932a37e737c925
  Stored in directory: /root/.cache/pip/wheels/01/ce/8f/

In [1]:
!pip install py-solc-x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Imports

In [7]:
import pandas as pd
import numpy as np
import json
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from flask import Flask, Response, request, jsonify
from web3 import Web3, HTTPProvider, EthereumTesterProvider
from solcx import install_solc, compile_source

install_solc(version='latest')

Version('0.8.17')

# How to share configuration

Decide on the configuration that should be used by owner ,user, as well as splitting the data to be used by each.

#### Data

In [3]:
df = pd.read_csv('uci_malware_detection.csv')

In [4]:
y = df["Label"]
X = df.drop("Label", axis=1)

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=42)

#### Setting up configurations to access smart contract

Make sure you are connected to the blockchain (`w3.isConnected()` should return `True`)

In [5]:
w3 = Web3(EthereumTesterProvider())
w3.isConnected()

True

In [8]:
compiled_sol = compile_source(
    '''
    pragma solidity >0.5.0;

    contract ModelParameters {
        string public encoded;

        constructor() public {
            encoded = '';
        }

        function setModelParameters(string memory _encoded) public {
            encoded = _encoded;
        }

        function getModelParameters() view public returns (string memory) {
            return encoded;
        }
    }
    ''',
    output_values=['abi', 'bin']
)

contract_id, contract_interface = compiled_sol.popitem()
bytecode = contract_interface['bin']
abi = contract_interface['abi']

ModelParameters = w3.eth.contract(abi=abi, bytecode=bytecode)

tx_hash = ModelParameters.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

ml_contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

/usr/local/lib/python3.7/dist-packages/eth_abi/codec.py:90: DeprecationWarning: abi.encode_abi() and abi.encode_abi_packed() are deprecated and will be removed in version 4.0.0 in favor of abi.encode() and abi.encode_packed(), respectively
  category=DeprecationWarning,


# Section 1: Miner

This is the part that will be used by the miner. Here, the model is trained on private data, and the model parameters are uploaded to the smart contract. We should ensure that the address of the user uploading this data is the same as the owner of the smart contract.

### Training

In [9]:
cls = LogisticRegression()
cls.fit(X_train,y_train)

LogisticRegression()

In [10]:
predictions = cls.predict(X_test);

accuracy = np.sum(predictions == y_test) / y_test.shape[0] * 100
conf_matrix = confusion_matrix(predictions, y_test)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1]) * 100

print(conf_matrix)
print("Accuracy: {0:.6f}%".format(accuracy))
print("Precision: {0:.6f}%".format(precision))

[[57  1]
 [ 1 16]]
Accuracy: 97.333333%
Precision: 98.275862%


### Encode model parameters to json

In [11]:
encoded = json.dumps((cls.coef_.tolist(), cls.intercept_.tolist(), cls.classes_.tolist()))
encoded

'[[[-2.4489667504150466e-06, 0.0, -0.009180107023493629, 0.009177658056743216, -0.009180107023493629, 0.0, -0.009180107023493629, 0.0, -0.009180107023493629, 0.0, -0.07864359632019667, 0.0, 0.06618083460446468, 0.0, -0.06075611942196961, 0.0, 0.004764584219741587, 0.0, 0.28565366943949855, -0.3928409801894102, 0.15502346654336316, 0.0, 0.06709490791826057, 0.08792855862510263, 0.06709490791826057, -0.09819354919879178, 0.06709490791826057, 0.0, 0.15905875813496206, -0.1590612071017125, 0.15502346654336316, 0.0, 0.073545132874597, 0.0, 0.006083865640033722, 0.09242802300114442, -0.10160813002463805, 0.09242802300114442, -0.10160813002463805, 0.00016070178046719713, 0.0669342061377934, -0.1092356261523015, 0.0669342061377934, -0.07611431316128701, 0.0669342061377934, 0.08149383315723732, 0.18592066433841217, 0.024982255255052266, -0.2685168917673318, 0.25933678474383814, -0.2859004720606314, 0.1804390202229301, -0.26941510512244105, 0.00016070178046719713, 0.18663308138873613, 0.03040679

### Upload to Smart Contract

In [12]:
w3.eth.defaultAccount = w3.eth.accounts[0]

tx_hash = ml_contract.functions.setModelParameters(encoded)
tx_hash = tx_hash.transact()

# Wait for transaction to be mined...
w3.eth.waitForTransactionReceipt(tx_hash)

/usr/local/lib/python3.7/dist-packages/web3/eth.py:169: DeprecationWarning: defaultAccount is deprecated in favor of default_account
  category=DeprecationWarning,
/usr/local/lib/python3.7/dist-packages/eth_abi/codec.py:90: DeprecationWarning: abi.encode_abi() and abi.encode_abi_packed() are deprecated and will be removed in version 4.0.0 in favor of abi.encode() and abi.encode_packed(), respectively
  category=DeprecationWarning,
/usr/local/lib/python3.7/dist-packages/web3/_utils/decorators.py:50: DeprecationWarning: waitForTransactionReceipt is deprecated in favor of wait_for_transaction_receipt
  category=DeprecationWarning)


AttributeDict({'blockHash': HexBytes('0x4dad2114d818aa518f6a4eb1079f6183bb9167ed4389dafc941a80825a5ab3c8'),
 'blockNumber': 2,
 'contractAddress': None,
 'cumulativeGasUsed': 7925985,
 'effectiveGasPrice': 1000000000,
 'from': '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 'gasUsed': 7925985,
 'logs': [],
 'state_root': b'\x01',
 'status': 1,
 'to': '0xF2E246BB76DF876Cef8b38ae84130F4F55De395b',
 'transactionHash': HexBytes('0x6a52622f5f246b308c6f006f6f140f1ce783e4499c6438b4d1e8016b5124669e'),
 'transactionIndex': 0,
 'type': '0x2'})

# Module 2: User

Now we will check how the user can retrieve the model parameters from the smart contract, and create a logistic regression model based on them. 

### Retrieving data from smart contract and decoding

In [13]:
w3.eth.defaultAccount = w3.eth.accounts[1]

encoded_parameters = ml_contract.functions.getModelParameters().call()
encoded_parameters

/usr/local/lib/python3.7/dist-packages/eth_abi/codec.py:194: DeprecationWarning: abi.decode_abi() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  category=DeprecationWarning


'[[[-2.4489667504150466e-06, 0.0, -0.009180107023493629, 0.009177658056743216, -0.009180107023493629, 0.0, -0.009180107023493629, 0.0, -0.009180107023493629, 0.0, -0.07864359632019667, 0.0, 0.06618083460446468, 0.0, -0.06075611942196961, 0.0, 0.004764584219741587, 0.0, 0.28565366943949855, -0.3928409801894102, 0.15502346654336316, 0.0, 0.06709490791826057, 0.08792855862510263, 0.06709490791826057, -0.09819354919879178, 0.06709490791826057, 0.0, 0.15905875813496206, -0.1590612071017125, 0.15502346654336316, 0.0, 0.073545132874597, 0.0, 0.006083865640033722, 0.09242802300114442, -0.10160813002463805, 0.09242802300114442, -0.10160813002463805, 0.00016070178046719713, 0.0669342061377934, -0.1092356261523015, 0.0669342061377934, -0.07611431316128701, 0.0669342061377934, 0.08149383315723732, 0.18592066433841217, 0.024982255255052266, -0.2685168917673318, 0.25933678474383814, -0.2859004720606314, 0.1804390202229301, -0.26941510512244105, 0.00016070178046719713, 0.18663308138873613, 0.03040679

### Creating model based on parameters retrieved

In [14]:
decoded_parameters = json.loads(encoded_parameters)
cls_global = LogisticRegression()

cls_global.coef_ = np.array(decoded_parameters[0])
cls_global.intercept_ = np.array(decoded_parameters[1])
cls_global.classes_ = np.array(decoded_parameters[2])

### Performing Prediction

In [15]:
predictions = cls_global.predict(X_test);
result = pd.DataFrame(np.vstack((predictions, y_test)).T,columns=['Predicted Outcomes','Actual Outcomes'])
result.head()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


,Predicted Outcomes,Actual Outcomes
0,malicious,malicious
1,non-malicious,non-malicious
2,non-malicious,non-malicious
3,malicious,malicious
4,non-malicious,non-malicious


### Showing resulting metrics

In [16]:
accuracy = np.sum(predictions == y_test) / y_test.shape[0] * 100
conf_matrix = confusion_matrix(predictions, y_test)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1]) * 100

print(conf_matrix)
print("Accuracy: {0:.6f}%".format(accuracy))
print("Precision: {0:.6f}%".format(precision))

[[57  1]
 [ 1 16]]
Accuracy: 97.333333%
Precision: 98.275862%
